# Projet maison n° 4

In [442]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas, pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [443]:
# names us

def df_names_us():
    #pour définir les titres des différents fichiers.txt à prendre en considération 
    years = list(range(1880,2020,1))
    
   
    #Récupération des fichiers 
    
    dfs_list = [] #la méthode pd.concat prend en argument une liste 
    for year in years: 
        df = pd.read_csv("yob" + str(year) + ".txt", names =['name', 'gender', 'births'])
        df.insert(0,'year', year)
        dfs_list.append(df)
    
    #Concaténation 
    df_res = pd.concat(dfs_list, ignore_index = True)
    return df_res

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [444]:
# names fr
def df_names_fr():
    
    #récupération des données 
    doc_name = 'nat2019.csv'
    df = pd.read_csv(doc_name, sep = ';')
    
    #Colonnes de df
    df.columns = ['gender','name', 'year','births']
    df = df.reindex(columns=['year','name', 'gender', 'births'])
    
    # même dtypes =>changement du type de year et de gender
    
    index_XXXX = df[ df['year'] == 'XXXX'].index
    #suppression des lignes avec une année XXXX
    df.drop(index_XXXX , inplace=True)
    
    df = df.astype({'year':'int64'}) 
     
    #Même valeur pour la colonne gender
    df = df.astype({'gender':'str'}) 
    mapping = {'2': 'F', '1' : 'M'}
    df['gender'] = df['gender'].map(mapping)
    df = df.astype({'gender':'object'}) 
    
    #Suppression des prénoms de plus de 2 caractères
    df = df.astype({'name':'str'}) 
    index_name_to_del = df[df['name'].map(len) < 2].index
    df.drop(index_name_to_del , inplace=True)
    
    
    #suppression des prénoms rares
    index_rare_name = df[df['name'] == '_PRENOMS_RARES'].index
    df.drop(index_rare_name , inplace=True)
    
    #Casse des prénoms
    df['name'] = df['name'].apply(lambda x : x.capitalize())
    df = df.astype({'name':'object'}) 
    
    #Tri 
    df_res = df.sort_values(by = ['year', 'gender', 'births', 'name'], ascending = [True, True, False, True])
    
    #Reset de l'index
    df_res.reset_index(drop = True, inplace = True) 
    #changement directement dans le dataframe
    #ici ne renvoie pas de data frame avec une colonne en plus et ne garde pas les anciennes positions
    
    
    return df_res

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [445]:
# taux de change
def df_taux_change(devises):
    
    df_res = pd.DataFrame()
    df_temp = pd.DataFrame()
    #récupération des données 
    # travail effectué sur les données disponibles au 25 octobre 

    doc_name = 'Webstat_Export_20201025.csv'
    df = pd.read_csv(doc_name, sep = ';')
    
   

    #liste des codes ISO
    temp = df.loc[1]
    list_temp = temp [1: len(temp)] 
    list_ISO_code = ['Date']
    for x in list_temp: 
        list_ISO_code.append(x[-4:-1])
        
    #modification du dataframe
    df.drop(df.index[0:5],0,inplace=True)

    df.columns = list_ISO_code
    
    #Transformation de la colonne Date / Index

    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df.set_index('Date')
    
    
    cpt =0
    
    #récupération des différentes devises et inclusion dans un nouveau dataFrame
    for devise in devises: 
        df_temp.insert(loc = cpt, column = devise, value = df[devise])
        cpt = cpt+1
    
    #Mise en place de l'index Date
    df_temp.index = df['Date']
    
    print(df_temp)
    print(df_temp.dtypes)
    
    #Nettoyage de la nouvelle base de données
    
    
    for devise in devises: 
        df_temp = df_temp.astype({devise:'str'}) 
        df_temp[devise] = [x.replace(",", ".") for x in df_temp[devise]]
        index = df_temp[df_temp[devise] == '-'].index
        df_temp.drop(index , inplace=True)
        df_temp = df_temp.astype({devise:'float64'}) 
    df_res = df_temp.dropna()
    
    
    return df_res

### Tests

In [446]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [447]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [448]:
run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... 

               CHF      GBP     USD
Date                               
2020-10-23  1,0715  0,90675  1,1856
2020-10-22  1,0726  0,90273  1,1821
2020-10-21  1,0715  0,90754  1,1852
2020-10-20  1,0724  0,91329   1,181
2020-10-19  1,0724  0,90588  1,1785
...            ...      ...     ...
1999-01-08  1,6138   0,7094  1,1659
1999-01-07  1,6165  0,70585  1,1632
1999-01-06  1,6116   0,7076  1,1743
1999-01-05  1,6123   0,7122   1,179
1999-01-04  1,6168   0,7111  1,1789

[7964 rows x 3 columns]
CHF    object
GBP    object
USD    object
dtype: object


ok

----------------------------------------------------------------------
Ran 3 tests in 3.389s

OK
